In [1]:
collection = [
    ("Document1", "This is a sample"),
    ("Document2", "This is another sample")
]

In [2]:
# in-memory
# 전체 색인어 목록(Dictionary)
# {단어1:포스팅위치-fptr, 단어2:포스팅위치, ...} fseek(fptr), fread(4byters*4)
globalLexicon = dict()
# 전체 문서 목롤(Dictionary)
# [0:문서1-이름, 1:문서2-이름, ...]
globalDocument = list()
# disk
# 사전에 있는 색인어 중, 어는 문서에서, 몇번 나탔는지
# Tuple(lexiconIdx, documentIdx, freq, 다음포스틩위치-ptr)
globalPosting = list()

for (docName, docContent) in collection:
#     0: Document1
#         len = 1
#     1: document2
    docIdx = len(globalDocument) # Crawling - Duplicated X
    globalDocument.append(docName)
    
    # 문서 1개에 대해서만,
    # {색인어1의 idx:빈도, 색인어2:빈도, ...}
    localPosting = dict()
    
    for token in docContent.lower().split():
        # Local에 대해서, 없으면 추가
        if token not in localPosting.keys():
            localPosting[token] =1
        # 있으면, 빈도 증가
        else:
            localPosting[token] += 1
            
    # Global Merge
    for token, freq in localPosting.items(): 

        if token not in globalLexicon.keys(): #         if 색인어 목록 X -> 새로운 단어 추가
            
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = (lexiconIdx, docIdx, freq, -1) # -1은 마지막 위치라는 것을 가리키기 위해
            globalPosting.append(postingData)
            globalLexicon[token] = postingIdx
            # 글로발(사전)에서 위치만 가져온, 나중에 posting 위치를 추가
        else: #         else 기존단어의 idx 가져오기
            lexiconIdx = list(globalLexicon.keys()).index(token)
            postingIdx = len(globalPosting)
            oldPostingIdx = globalLexicon[token]
            postingData = (lexiconIdx, docIdx, freq, oldPostingIdx) 
            globalPosting.append(postingData)
            globalLexicon[token] = postingIdx

    print(docName)

Document1
Document2


In [3]:
globalLexicon

{'this': 4, 'is': 5, 'a': 2, 'sample': 7, 'another': 6}

In [4]:
globalPosting

[(0, 0, 1, -1),
 (1, 0, 1, -1),
 (2, 0, 1, -1),
 (3, 0, 1, -1),
 (0, 1, 1, 0),
 (1, 1, 1, 1),
 (4, 1, 1, -1),
 (3, 1, 1, 3)]

In [8]:
globalPosting[globalLexicon['sample']]

(3, 1, 1, 3)

In [9]:
globalPosting[globalPosting[globalLexicon['sample']][3]]   # 다음 주소가 "-1" 일때 까지 반복해서 찾음

(3, 0, 1, -1)

In [15]:
collection = [
    ('Document1', 'This is a a a a a a a a a a sample'),
    ('Document2', 'This is a sample'),
]

In [16]:
# indexTerm
for indexTerm, postingIdx in globalLexicon.items():
    print(indexTerm)
    
    while True:
        if postingIdx == -1:
            break
            
        postingData = globalPosting[postingIdx]
        print(" DocName:{0} - TermFreq:{1} - Next:{2} ".format(
             globalDocument[postingData[1]],
             postingData[2],
             postingData[3]
        ))
        
        postingIdx = postingData[3]
        
    print()

this
 DocName:Document2 - TermFreq:1 - Next:0 
 DocName:Document1 - TermFreq:1 - Next:-1 

is
 DocName:Document2 - TermFreq:1 - Next:1 
 DocName:Document1 - TermFreq:1 - Next:-1 

a
 DocName:Document1 - TermFreq:1 - Next:-1 

sample
 DocName:Document2 - TermFreq:1 - Next:3 
 DocName:Document1 - TermFreq:1 - Next:-1 

another
 DocName:Document2 - TermFreq:1 - Next:-1 



In [17]:
from math import log10, log2
# TF
def binaryTF(freq):
    if freq > 0:
        return 1
    else:
        return 0
    
def rawTF(freq):
    return freq

def basicTF(freq, totalFreq):
    return freq/totalFreq

def logTF(freq):
    if freq > 0:
        return 1 + log10(freq)
    else:
        return 0
    
def doubleNormalTF(K, freq, maxFreq):
    return K + ((1-K) * (freq/ maxFreq))

In [18]:
for (docName, docContent) in collection:

    localPosting = dict()
    
    for term in docContent.lower().split():
        # Local에 대해서, 없으면 추가
        if term not in localPosting.keys():
            localPosting[term] =1
        # 있으면, 빈도 증가
        else:
            localPosting[term] +=1
            
    maxFreq = max(localPosting.values())
    totalCount = sum(localPosting.values())
    
    for token, freq in localPosting.items():
        print(token)   
        print("1. Binary: {0}".format(binaryTF(freq)))
        print("2. Raw : {0}".format(rawTF(freq)))
        print("3. Basic : {0}".format(basicTF(freq,totalCount)))
        print("4. Log : {0}".format(logTF(freq)))
        print("5. DoubleNormalization : {0}".format(doubleNormalTF(0,freq, maxFreq )))   #
        print("6. DoubleNormalization : {0}".format(doubleNormalTF(0.5,freq, maxFreq )))
        print()
        

this
1. Binary: 1
2. Raw : 1
3. Basic : 0.07692307692307693
4. Log : 1.0
5. DoubleNormalization : 0.1
6. DoubleNormalization : 0.55

is
1. Binary: 1
2. Raw : 1
3. Basic : 0.07692307692307693
4. Log : 1.0
5. DoubleNormalization : 0.1
6. DoubleNormalization : 0.55

a
1. Binary: 1
2. Raw : 10
3. Basic : 0.7692307692307693
4. Log : 2.0
5. DoubleNormalization : 1.0
6. DoubleNormalization : 1.0

sample
1. Binary: 1
2. Raw : 1
3. Basic : 0.07692307692307693
4. Log : 1.0
5. DoubleNormalization : 0.1
6. DoubleNormalization : 0.55

this
1. Binary: 1
2. Raw : 1
3. Basic : 0.25
4. Log : 1.0
5. DoubleNormalization : 1.0
6. DoubleNormalization : 1.0

is
1. Binary: 1
2. Raw : 1
3. Basic : 0.25
4. Log : 1.0
5. DoubleNormalization : 1.0
6. DoubleNormalization : 1.0

a
1. Binary: 1
2. Raw : 1
3. Basic : 0.25
4. Log : 1.0
5. DoubleNormalization : 1.0
6. DoubleNormalization : 1.0

sample
1. Binary: 1
2. Raw : 1
3. Basic : 0.25
4. Log : 1.0
5. DoubleNormalization : 1.0
6. DoubleNormalization : 1.0



In [19]:
def unaryIDF():
    return 1

def basicIDF(N, df):
    return log10(N/df)

def smoothingIDF(N, df):
    return log10((1+N)/df)  #  < 0 (N=df) 이 경우 -값이 나온다

def probabilityIDF(N, df):
    return log10((N-df+1)/df) #  절반보단 많이 나오면 - 절반보다 적개나오면...+

In [20]:
globalLexicon

{'this': 4, 'is': 5, 'a': 2, 'sample': 7, 'another': 6}

In [21]:
N = len(globalDocument)

for term, ptr in globalLexicon.items():
    
    df = 0
    
    while True:
        if ptr == -1:
            break
            
        df += 1
        
        postingData = globalPosting[ptr]
        ptr = postingData[3]
    print(term)
#     print("{0}. {1}".format(term, df))
    print("1. UnaryIDF: {0}".format(unaryIDF()))
    print("2. BasicIDF : {0}".format(basicIDF(N, df)))
    print("3. SmoothingIDF : {0}".format(smoothingIDF(N,df)))
    print("4. ProbabilityIDF : {0}".format(probabilityIDF(N, df)))
  

this
1. UnaryIDF: 1
2. BasicIDF : 0.0
3. SmoothingIDF : 0.17609125905568124
4. ProbabilityIDF : -0.3010299956639812
is
1. UnaryIDF: 1
2. BasicIDF : 0.0
3. SmoothingIDF : 0.17609125905568124
4. ProbabilityIDF : -0.3010299956639812
a
1. UnaryIDF: 1
2. BasicIDF : 0.3010299956639812
3. SmoothingIDF : 0.47712125471966244
4. ProbabilityIDF : 0.3010299956639812
sample
1. UnaryIDF: 1
2. BasicIDF : 0.0
3. SmoothingIDF : 0.17609125905568124
4. ProbabilityIDF : -0.3010299956639812
another
1. UnaryIDF: 1
2. BasicIDF : 0.3010299956639812
3. SmoothingIDF : 0.47712125471966244
4. ProbabilityIDF : 0.3010299956639812
